# Excel DataFrame Processor - Jupyter Notebook Example

This notebook demonstrates how to use the Excel DataFrame Processor in Jupyter notebooks for data analysis and visualization.

## Features Covered:
- 📊 Loading Excel files programmatically
- 🔍 Executing SQL queries on Excel data
- 🎨 Using magic commands for convenient querying
- 📈 Data visualization with matplotlib and seaborn
- 📤 Exporting results to CSV
- 🔗 Joining data from multiple Excel files

## Setup and Installation

First, make sure you have the Excel DataFrame Processor installed and sample data created:

In [ ]:
# Install required packages (run this if needed)
# !pip install pandas openpyxl matplotlib seaborn plotly

# Create sample data (run this if sample_data directory doesn't exist)
# !python create_sample_data.py

## Import Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

# Import Excel DataFrame Processor
from excel_processor.notebook import ExcelProcessor

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

## Method 1: Programmatic Interface

### Initialize the Excel Processor

In [ ]:
# Initialize the Excel processor with sample data directory
excel_processor = ExcelProcessor(db_directory='sample_data', memory_limit_mb=512)

print("✅ Excel DataFrame Processor initialized!")
print(f"📁 Database directory: {excel_processor.db_directory}")

### Explore Available Data

In [ ]:
# Show all available Excel files and sheets
db_info = excel_processor.show_db()

In [ ]:
# Load all files into memory for faster querying
load_info = excel_processor.load_db()

### Basic SQL Queries

In [ ]:
# Query 1: View all employees
employees = excel_processor.query("SELECT * FROM employees.staff")
print(f"📊 Total employees: {len(employees)}")

In [ ]:
# Query 2: High earners only
high_earners = excel_processor.query(
    "SELECT name, department, salary FROM employees.staff WHERE salary > 70000 ORDER BY salary DESC"
)
print(f"💰 High earners (>$70k): {len(high_earners)}")

In [ ]:
# Query 3: Department summary
dept_summary = excel_processor.query(
    "SELECT department FROM employees.staff"
)

# Since GROUP BY is not fully implemented yet, let's use pandas for aggregation
dept_stats = employees.groupby('department').agg({
    'salary': ['count', 'mean', 'min', 'max'],
    'age': 'mean'
}).round(2)

dept_stats.columns = ['employee_count', 'avg_salary', 'min_salary', 'max_salary', 'avg_age']
print("📈 Department Statistics:")
display(dept_stats)

### Working with Multiple Files

In [ ]:
# Query orders data
orders = excel_processor.query("SELECT * FROM orders.sales_data")
print(f"📦 Total orders: {len(orders)}")

In [ ]:
# Query products data
products = excel_processor.query("SELECT * FROM products.catalog")
inventory = excel_processor.query("SELECT * FROM products.inventory")

print(f"🛍️ Products in catalog: {len(products)}")
print(f"📦 Inventory records: {len(inventory)}")

### Data Analysis and Visualization

In [ ]:
# Salary distribution by department
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.boxplot(data=employees, x='department', y='salary')
plt.title('Salary Distribution by Department')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
dept_counts = employees['department'].value_counts()
plt.pie(dept_counts.values, labels=dept_counts.index, autopct='%1.1f%%')
plt.title('Employee Distribution by Department')

plt.tight_layout()
plt.show()

In [ ]:
# Sales analysis
plt.figure(figsize=(12, 8))

# Convert order_date to datetime if it's not already
orders['order_date'] = pd.to_datetime(orders['order_date'])

plt.subplot(2, 2, 1)
monthly_sales = orders.groupby(orders['order_date'].dt.to_period('M'))['amount'].sum()
monthly_sales.plot(kind='bar')
plt.title('Monthly Sales')
plt.xticks(rotation=45)

plt.subplot(2, 2, 2)
region_sales = orders.groupby('region')['amount'].sum()
region_sales.plot(kind='bar', color='skyblue')
plt.title('Sales by Region')

plt.subplot(2, 2, 3)
status_counts = orders['status'].value_counts()
plt.pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%')
plt.title('Order Status Distribution')

plt.subplot(2, 2, 4)
plt.scatter(orders['order_date'], orders['amount'], alpha=0.6)
plt.title('Order Amount Over Time')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

### Interactive Plotly Visualizations

In [ ]:
# Interactive salary vs age scatter plot
fig = px.scatter(
    employees, 
    x='age', 
    y='salary', 
    color='department',
    size='salary',
    hover_data=['name'],
    title='Employee Salary vs Age by Department'
)
fig.show()

In [ ]:
# Interactive sales timeline
fig = px.line(
    orders.sort_values('order_date'), 
    x='order_date', 
    y='amount',
    color='region',
    title='Sales Timeline by Region',
    hover_data=['customer', 'status']
)
fig.show()

## Method 2: Magic Commands

Load the Excel magic commands for even more convenient usage:

In [ ]:
# Load Excel magic commands
%load_ext excel_processor.notebook

print("✨ Excel magic commands loaded!")
print("Available commands:")
print("  %excel_init --db <directory>")
print("  %excel_show_db")
print("  %excel_load_db")
print("  %excel_memory")
print("  %%excel_sql")

In [ ]:
# Initialize with magic command
%excel_init --db sample_data --memory-limit 512

In [ ]:
# Show database contents
%excel_show_db

In [ ]:
# Load all files
%excel_load_db

In [ ]:
# Execute SQL with cell magic
%%excel_sql
SELECT name, department, salary 
FROM employees.staff 
WHERE salary > 75000 
ORDER BY salary DESC

In [ ]:
# Another SQL query with magic
%%excel_sql
SELECT * FROM products.catalog WHERE price > 500

In [ ]:
# Check memory usage
%excel_memory

## Advanced Analysis: Combining Data Sources

In [ ]:
# Since complex joins aren't implemented yet, let's use pandas to combine data
# This simulates what the full SQL engine would do

# Employee sales performance
employee_sales = orders.groupby('employee_id').agg({
    'order_id': 'count',
    'amount': ['sum', 'mean']
}).round(2)

employee_sales.columns = ['order_count', 'total_sales', 'avg_order_value']
employee_sales = employee_sales.reset_index()

# Merge with employee data
employee_performance = employees.merge(
    employee_sales, 
    left_on='id', 
    right_on='employee_id', 
    how='left'
).fillna(0)

print("👥 Employee Sales Performance:")
display(employee_performance[['name', 'department', 'salary', 'order_count', 'total_sales', 'avg_order_value']].head(10))

In [ ]:
# Visualize employee performance
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Sales vs Salary
axes[0, 0].scatter(employee_performance['salary'], employee_performance['total_sales'], 
                   c=employee_performance['order_count'], cmap='viridis', alpha=0.7)
axes[0, 0].set_xlabel('Salary')
axes[0, 0].set_ylabel('Total Sales')
axes[0, 0].set_title('Sales Performance vs Salary')

# Department performance
dept_performance = employee_performance.groupby('department')[['total_sales', 'order_count']].sum()
dept_performance['total_sales'].plot(kind='bar', ax=axes[0, 1])
axes[0, 1].set_title('Total Sales by Department')
axes[0, 1].tick_params(axis='x', rotation=45)

# Order count distribution
axes[1, 0].hist(employee_performance['order_count'], bins=10, alpha=0.7, color='skyblue')
axes[1, 0].set_xlabel('Number of Orders')
axes[1, 0].set_ylabel('Number of Employees')
axes[1, 0].set_title('Distribution of Orders per Employee')

# Top performers
top_performers = employee_performance.nlargest(5, 'total_sales')
axes[1, 1].barh(top_performers['name'], top_performers['total_sales'])
axes[1, 1].set_xlabel('Total Sales')
axes[1, 1].set_title('Top 5 Sales Performers')

plt.tight_layout()
plt.show()

## Export Results

In [ ]:
# Export high earners to CSV using SQL syntax
high_earners_export = excel_processor.query(
    "SELECT name, department, salary FROM employees.staff WHERE salary > 70000 > high_earners.csv",
    display_result=False
)

In [ ]:
# Export employee performance analysis
employee_performance.to_csv('employee_performance_analysis.csv', index=False)
print("✅ Exported employee performance analysis to employee_performance_analysis.csv")

In [ ]:
# Export department summary
dept_stats.to_csv('department_summary.csv')
print("✅ Exported department summary to department_summary.csv")

## Memory Management

In [ ]:
# Check current memory usage
memory_info = excel_processor.get_memory_usage()

print("💾 Memory Usage Summary:")
print(f"  Total: {memory_info['total_mb']:.2f} MB")
print(f"  Limit: {memory_info['limit_mb']:.2f} MB")
print(f"  Usage: {memory_info['usage_percent']:.1f}%")
print(f"  Files loaded: {len(memory_info['files'])}")

if memory_info['files']:
    print("\n📄 Memory usage by file:")
    for file_name, usage in memory_info['files'].items():
        print(f"  {file_name}: {usage:.2f} MB")

## Summary and Next Steps

This notebook demonstrated:

✅ **Programmatic Interface**: Using `ExcelProcessor` class for direct Python integration  
✅ **Magic Commands**: Convenient `%%excel_sql` magic for inline SQL queries  
✅ **Data Analysis**: Combining Excel data with pandas for advanced analytics  
✅ **Visualization**: Creating charts and plots with matplotlib, seaborn, and plotly  
✅ **Export Capabilities**: Saving results to CSV files  
✅ **Memory Management**: Monitoring and controlling memory usage  

### Future Enhancements:
- Full JOIN support in SQL queries
- GROUP BY and aggregate functions in SQL
- More advanced filtering and sorting options
- Integration with other data sources
- Real-time data updates and monitoring

### Try These Next:
1. Create your own Excel files and analyze them
2. Combine multiple data sources for comprehensive analysis
3. Build interactive dashboards with plotly
4. Automate reporting workflows
5. Integrate with machine learning pipelines

Happy analyzing! 🚀📊